In [8]:
import sqlite3
# 连接到 SQLite 数据库
def get_sqlite_schema_str(db_id: str):
    db_path = "../dataset/spider/database/{db_id}/{db_id}.sqlite".format(db_id=db_id)
    conn = sqlite3.connect(db_path)

    # 创建游标对象
    cursor = conn.cursor()

    # 执行查询获取表的模式信息
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table'")

    # 获取查询结果
    results = cursor.fetchall()

    # # 打印表的模式信息
    # # schema = "```sql\n"
    schema=""
    for result in results:
        schema += result[0].lower() + ";\n"
    # schema+="```\n"
    # 关闭连接
    conn.close()

    return schema

In [10]:
import json


Spider_Dataset_Path = "../dataset/spider/"
Dev_Json_Path = Spider_Dataset_Path + "train_spider.json"
Database_Path = Spider_Dataset_Path + "database/"


def build_instruction_prompt(schema: str, instruction: str):
    return """
Given the following SQLite database schema:
{schema}
### Instruction:
Complete the SQLite SQL query only to answer this question and with no explanation:
{instruction}
### Response:
""".format(
        schema=schema, instruction=instruction.strip()
    ).lstrip()


output = []
with open(Dev_Json_Path, "r") as dataset_file:
    i = 0
    dataset = json.load(dataset_file)
    for case in dataset:
        schema = get_sqlite_schema_str(case["db_id"])
        prompt = build_instruction_prompt(schema, case["question"])

        train_data = {"instruction": prompt, "output": case["query"]}
        # print(train_data["instruction"])
        output.append(train_data)
        # break
with open("dataset.json", "w") as output_file:
    json.dump(output, output_file)